# Deceleration, Braking, and Regenerative Braking

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 100

## 1 Load Data

In [48]:
# file names
read_path = 'data/2-mapmatched/matched/'
traj_files = os.listdir(read_path)

# read trajectory
trajs = {}
for traj_file in traj_files:
    key = traj_file.split('.')[0]
    trajs[key] = pd.read_csv(read_path + traj_file)

## 2 Braking Events

How to detect braking, regenerative braking from deceleration?
- Criteria for deceleration:
  - `Acc` < 0
- Criteria for braking:
  - `Tractive Power` < 0
- Criteria for regenerative braking (EV):
  - `Tractive Power` < 0
  - `Brake Pad Signal` > static signal

In [49]:
# initialize
decel_set = pd.DataFrame({})
# iterate
for key, traj_raw in trajs.items():
    traj = traj_raw.copy()
    traj = traj[['sec of day', 'lon', 'lat', 'speed[km/h]', 'acc[m/s2]', 'trac_power[kW]', 'pad_pressure', 'true_north[D]', 'grade[D]', 'link_id', 'highway']]
    # detect deceleration
    decel = traj[traj['acc[m/s2]'] < 0].copy()
    # detect braking
    decel['braking'] = decel['trac_power[kW]'].apply(lambda x: 1 if x < 0 else 0)
    # set event_id
    sec_discontinuity = decel['sec of day'].diff() != 1
    decel['event_id'] = sec_discontinuity.cumsum()
    # veh type and driver
    veh, driver = key.split("-")
    decel['model'] = veh
    decel['driver'] = driver
    # merge
    decel_set = pd.concat([decel_set, decel])

decel_set.reset_index(drop=True, inplace=True)

## 3 Vehicle Info

In [50]:
veh_info = pd.read_excel('data/veh_info.xlsx')
decel_veh_set = pd.merge(decel_set, veh_info[["model", "brand", "class", "shape", "weight/kg"]], on="model", how="left")
decel_veh_set['fuel'] = decel_veh_set['model'].apply(lambda x: 'EV' if x in ['Ei5', 'ModelY'] else 'ICEV')
decel_veh_set.head()

,sec of day,lon,lat,speed[km/h],acc[m/s2],trac_power[kW],pad_pressure,true_north[D],grade[D],link_id,highway,braking,event_id,model,driver,brand,class,shape,weight/kg,fuel
0,52729.0,118.822190,31.896260,54.387636,-2.770374,-66.506187,21.0,85.409020,-0.004052,749,secondary,1,1,Avalon,DRIVER1,Toyota,B,sedan,1570,ICEV
1,52730.0,118.822303,31.896270,44.414291,-1.542779,-29.200490,10.0,85.514706,-0.000726,749,secondary,1,1,Avalon,DRIVER1,Toyota,B,sedan,1570,ICEV
2,52731.0,118.822410,31.896278,38.860287,-0.322834,-3.481604,0.0,85.133412,0.002181,749,secondary,1,1,Avalon,DRIVER1,Toyota,B,sedan,1570,ICEV
3,52739.0,118.823447,31.896356,48.592204,-0.107373,0.970680,0.0,85.125913,-0.010726,749,secondary,0,2,Avalon,DRIVER1,Toyota,B,sedan,1570,ICEV
4,52740.0,118.823586,31.896365,48.205661,-0.389990,-5.454765,0.0,85.562774,-0.010775,749,secondary,1,2,Avalon,DRIVER1,Toyota,B,sedan,1570,ICEV


## 4 Regenerative Braking

In [52]:
decel_veh_set['reg'] = 0
decel_veh_set.loc[
    ((decel_veh_set['fuel'] == 'EV') &
    (decel_veh_set['braking'] == 1) &
    (decel_veh_set['pad_pressure'] < 5)),  # set 10 as the threshold of braking signal
    'reg'] = 1
decel_veh_set.head()

,sec of day,lon,lat,speed[km/h],acc[m/s2],trac_power[kW],pad_pressure,true_north[D],grade[D],link_id,...,braking,event_id,model,driver,brand,class,shape,weight/kg,fuel,reg
0,52729.0,118.822190,31.896260,54.387636,-2.770374,-66.506187,21.0,85.409020,-0.004052,749,...,1,1,Avalon,DRIVER1,Toyota,B,sedan,1570,ICEV,0
1,52730.0,118.822303,31.896270,44.414291,-1.542779,-29.200490,10.0,85.514706,-0.000726,749,...,1,1,Avalon,DRIVER1,Toyota,B,sedan,1570,ICEV,0
2,52731.0,118.822410,31.896278,38.860287,-0.322834,-3.481604,0.0,85.133412,0.002181,749,...,1,1,Avalon,DRIVER1,Toyota,B,sedan,1570,ICEV,0
3,52739.0,118.823447,31.896356,48.592204,-0.107373,0.970680,0.0,85.125913,-0.010726,749,...,0,2,Avalon,DRIVER1,Toyota,B,sedan,1570,ICEV,0
4,52740.0,118.823586,31.896365,48.205661,-0.389990,-5.454765,0.0,85.562774,-0.010775,749,...,1,2,Avalon,DRIVER1,Toyota,B,sedan,1570,ICEV,0


In [53]:
decel_veh_set = decel_veh_set[[
    'sec of day', 'lon', 'lat',
    'speed[km/h]', 'acc[m/s2]', 'trac_power[kW]', 'pad_pressure',
    'event_id', 'braking', 'reg',
    'true_north[D]', 'grade[D]', 'link_id', 'highway', 
    'driver', 'model', 'brand', 'class', 'shape', 'weight/kg', 'fuel'
]]
decel_veh_set.to_csv("data/3-braking/decel_set.csv", index=False)

---

## Consistensy of pad pressure and tractive power

In [54]:
decel_veh_set = pd.read_csv("data/3-braking/decel_set.csv")
decel_veh_set.head()

,sec of day,lon,lat,speed[km/h],acc[m/s2],trac_power[kW],pad_pressure,event_id,braking,reg,...,grade[D],link_id,highway,driver,model,brand,class,shape,weight/kg,fuel
0,52729.0,118.822190,31.896260,54.387636,-2.770374,-66.506187,21.0,1,1,0,...,-0.004052,749,secondary,DRIVER1,Avalon,Toyota,B,sedan,1570,ICEV
1,52730.0,118.822303,31.896270,44.414291,-1.542779,-29.200490,10.0,1,1,0,...,-0.000726,749,secondary,DRIVER1,Avalon,Toyota,B,sedan,1570,ICEV
2,52731.0,118.822410,31.896278,38.860287,-0.322834,-3.481604,0.0,1,1,0,...,0.002181,749,secondary,DRIVER1,Avalon,Toyota,B,sedan,1570,ICEV
3,52739.0,118.823447,31.896356,48.592204,-0.107373,0.970680,0.0,2,0,0,...,-0.010726,749,secondary,DRIVER1,Avalon,Toyota,B,sedan,1570,ICEV
4,52740.0,118.823586,31.896365,48.205661,-0.389990,-5.454765,0.0,2,1,0,...,-0.010775,749,secondary,DRIVER1,Avalon,Toyota,B,sedan,1570,ICEV


### Event-based

In [55]:
# all ICEV events
grouped_ICEV = decel_veh_set[decel_veh_set['fuel'] == 'ICEV'].groupby(by=['model', 'driver', 'event_id'])
decel_event_num = grouped_ICEV.__len__()

# events containing any braking
events_with_brakings = grouped_ICEV['braking'].any()
brake_event_num = events_with_brakings.sum()

# events containing any braking while with pad pressed
events_with_brakings_press = grouped_ICEV.apply(lambda group: ((group['braking'] == 1) & (group['pad_pressure'] >= 10)).any())
brake_press_event_num = events_with_brakings_press.sum()

print("Deceleration event: %d" % decel_event_num)
print("Braking event via tractive power: %d (%.1f%%)" % (brake_event_num, brake_event_num/decel_event_num*100))
print("Braking while pressing: %d (%.1f%%)" % (brake_press_event_num, brake_press_event_num/brake_event_num*100))

Deceleration event: 1305
Braking event via tractive power: 920 (70.5%)
Braking while pressing: 471 (51.2%)


### Trajectory-point-based

In [56]:
decel_num = decel_veh_set[decel_veh_set['fuel'] == 'ICEV'].shape[0]
braking_num = decel_veh_set[(decel_veh_set['fuel'] == 'ICEV') & (decel_veh_set['braking'] == 1)].shape[0]
pressing_num = decel_veh_set[(decel_veh_set['fuel'] == 'ICEV') & (decel_veh_set['pad_pressure'] >= 5)].shape[0]
braking_pressing_num = decel_veh_set[(decel_veh_set['fuel'] == 'ICEV') & (decel_veh_set['braking'] == 1) & (decel_veh_set['pad_pressure'] >= 5)].shape[0]
pressing_not_braking_num = decel_veh_set[(decel_veh_set['fuel'] == 'ICEV') & (decel_veh_set['braking'] == 0) & (decel_veh_set['pad_pressure'] >= 5)].shape[0]

print("Deceleration points: %d" % decel_num)
print("Braking points via tractive power: %d (%.1f%%)" % (braking_num, braking_num/decel_num*100))
print("Braking while pressing: %d (%.1f%%)" % (braking_pressing_num, braking_pressing_num/braking_num*100))
print("Pressing while not braking: %d" % (pressing_not_braking_num))

Deceleration points: 15207
Braking points via tractive power: 7548 (49.6%)
Braking while pressing: 5056 (67.0%)
Pressing while not braking: 3922
